In [ ]:
# #### #### #### #### 
# #### #### #### #### 
# #### #### #### #### 
# #### #### #### #### Código de clasificacion SIPRE (QUEJAS-DENUNICAS-UNIDAD)

### Ladino Álvarez Ricardo Arturo

# #### #### #### #### Librerias # #### #### #### #### 
# #### #### #### ####  Base

import re
import pickle
import numpy as np
import pandas as pd
import multiprocessing
from io import StringIO

## #### > NLP
import spacy
from nltk.corpus import stopwords
nlp = spacy.load('es_core_news_sm')
stopword = set(stopwords.words('spanish'))
from nltk.stem.snowball import SnowballStemmer
spanish_stemmer = SnowballStemmer('spanish')

## #### > NLP
import xgboost as xgb
from imblearn.over_sampling import SMOTEN
from sklearn import preprocessing, metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.model_selection import GridSearchCV, RepeatedKFold
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,mean_squared_error

## #### > Desbalance de clase
from collections import Counter
from imblearn.over_sampling import SMOTEN


## #### > Errores
import warnings
warnings.filterwarnings("ignore")

In [ ]:
## #### > Validación donde ambas bases tienen las mismas columnas
if list(Quejas.columns) == list(Denucias.columns):
    print("Ambas bases tienen los mismos nombres de columnas. Puedes concatenarlas.")
    # Concatenar si la validación es exitosa
    datos_concatenados = pd.concat([Quejas, Denucias], ignore_index=True)
    print("Concatenación exitosa.")
else:
    print("Las columnas no coinciden entre las bases. Revisa los nombres de las columnas.")
    print("Columnas en Quejas:", Quejas.columns.tolist())
    print("Columnas en Denuncias:", Denucias.columns.tolist())

In [ ]:
## #### > Función de representatividad

def Repre_clase(Clase_Size):
    if Clase_Size < 500:
        return 1.0  # 100%
    elif 500 <= Clase_Size <= 5000:
        return 0.8  # 80%
    else:
        return 0.5  # 50%
        
Muestras = Remuestreo['Tipo asunto'].value_counts().apply(Repre_clase)

## #### > Aplicar el muestreo por clase
Datos_Muestra = Remuestreo.groupby('Tipo asunto').apply(lambda x: x.sample(frac = Muestras[x.name],
                                                                            random_state = 42)).reset_index(drop = True)

print(Datos_Muestra.shape)
Datos_Muestra['Tipo asunto'].value_counts()

(73161, 7)


Tipo asunto
Denuncia    53102
Queja       20059
Name: count, dtype: int64

In [33]:
Tipo_asunto = pd.DataFrame(set(zip(Datos_Muestra['Tipo asunto'],
                                      Datos_Muestra['Tipo_asunto'])),
                              columns =['Queja_Denuncia', 'Code']).sort_values(by='Code', ascending = True)

UA_Asignada = pd.DataFrame(set(zip(Datos_Muestra['UA Asignada'],
                                      Datos_Muestra['UA_Asignada'])),
                              columns =['UA_Asignada', 'Code']).sort_values(by='Code', ascending = True)

### Limpieza de texto

In [ ]:
#### Función de Normalización del texto

### Limpieza de texto
def limpiar_texto(texto):
    # > 00 - StopWords
    # > 01 - Convertir a minúsculas
    # > 02 - Eliminar signos de puntuación y números
    # > 03 - Eliminar acentos y caracteres especiales
    # > 04 Eliminar stopwords
    # > 05 Eliminar palabras cortas
    # > 06 Corrección ortográfica
    # > 07 Lematización
    return (texto)

### Limpieza de múltiples columnas
def limpiar_columnas(df, columnas):
    for columna in columnas:
        df[columna] = df[columna].astype(str).fillna('')  # Convertir a string y manejar NaNs
        df[columna] = df[columna].apply(limpiar_texto)    # Aplicar la función de limpieza completa
    return (df)

In [35]:
Datos_Muestra_N = Datos_Muestra.copy()
Columnas_Limpia = ['Descripción de los Hechos'] 
Datos_Muestra_N = limpiar_columnas(Datos_Muestra_N, Columnas_Limpia)

### Prueba y entrenamiento

In [44]:
X_train, X_test, y_train, y_test = train_test_split(Datos_Muestra_N['Descripción de los Hechos'],
                                                    Datos_Muestra_N['Tipo_asunto'],
                                                    stratify = Datos_Muestra_N['Tipo_asunto'],
                                                    test_size = 0.60,
                                                    random_state = 42)

### Vector de aprendizaje + Bolsa de palabras

In [45]:
TFIDF = TfidfVectorizer(strip_accents = 'unicode', 
                        analyzer = 'word',
                        max_features = 10000,
                        norm='l2',
                        encoding = 'latin-1',
                        token_pattern = r'\w{1,}',
                        ngram_range = (1, 2), 
                        use_idf = True,
                        smooth_idf = True, 
                        sublinear_tf = True)

In [46]:
TDFT = TFIDF.fit(list(X_train) + list(X_test))
X_train_N =  TDFT.transform(X_train) 
X_test_N = TFIDF.transform(X_test)

In [39]:
pickle.dump(TDFT, open("Palabras_Tipo_Asunto.pkl", "wb"))

In [48]:
np.unique(y_train, return_counts=True)

(array([0, 1]), array([21241,  8023]))

In [49]:
### ================================= ================================= =============================== ###
### ========================================= QUEJAS & DENUNCIAS ====================================== ###
### ============================== FUNCIÓN DE APRENDIZAJE MAQUINA ===================================== ###

def Aprendizaje_Queja_Denuncia(X_train, X_test, y_train, y_test, Model):
    ''' Función de modelo de aprendizaje, que ejecuta lo siguiente:
        A) Se evaluan diferentes modelos : Random Forest, Naibe Bayes y XGBoost '''
    
    if Model == "Random_Forest":
        
        Ranf_clf = {'n_estimators': [64, 128, 256, 512, 1024],
                    'max_features': ['auto', 'sqrt', 'log2'],
                    'max_depth': [2, 4, 8, 16, 32, 64, 128],
                    'criterion': ['gini', 'entropy']
                   }
        
        estimador_RF = RandomForestClassifier(random_state = 42,
                                              n_jobs = multiprocessing.cpu_count()-1)
        
        RF_CECTI = GridSearchCV(estimator = estimador_RF,
                                param_grid = Ranf_clf,
                                cv = RepeatedKFold(n_splits = 3, 
                                                   n_repeats = 1, 
                                                   random_state = 42),
                                scoring = 'accuracy',
                                refit = True,
                                verbose = 0,
                                return_train_score = True)
        
        CECTI_RF = RF_CECTI.fit(X_train,y_train)
        
        Best_estimador = CECTI_RF.best_estimator_
        
        pickle.dump(Best_estimador, open("Estimador_RF_QD.pkl", "wb"))
        
        y_hat = Best_estimador.predict(X = X_test)
        
        RF_accuracy = metrics.accuracy_score(y_hat, y_test)
        
        print("----------------------------")
        print("----------------------------")
        print ("Random Forest > Accuracy: ", np.round((RF_accuracy*100), 3), '%')
        print("----------------------------")
        print('Reporte de clasificación: \n', classification_report(y_test, y_hat))
        
    elif Model == "Naibe_Bayes":
        
        n_classes = np.unique(y_train)
        
        params = {'alpha': [0.01, 0.1, 0.5, 1.0, 10.0],
                  'fit_prior': [True, False],
                  'class_prior': [None, [0.1,]* len(n_classes)]
                 }
        
        Esti_MNB = MultinomialNB()
        
        MNB_grid = GridSearchCV(Esti_MNB,
                                param_grid = params, 
                                n_jobs = multiprocessing.cpu_count()-1,
                                cv = RepeatedKFold(n_splits = 3, n_repeats = 1,
                                                   random_state = 42),
                                verbose =0 )
        
        CECTI_MNB = MNB_grid.fit(X_train,y_train)
        
        Best_estimador = CECTI_MNB.best_estimator_
        
        pickle.dump(Best_estimador, open("Estimador_NB_QD.pkl", "wb"))
        
        y_hat = Best_estimador.predict(X = X_test)
        
        NB_accuracy = metrics.accuracy_score(y_hat, y_test)
        
        print("----------------------------")
        print("----------------------------")
        print ("Naibe Bayes > Accuracy: ", np.round((NB_accuracy*100), 3), '%')
        print("----------------------------")
        print('Reporte de clasificación: \n', classification_report(y_test, y_hat))
    
    elif Model == "XGBoost":
        
        xgb_params = {'max_depth':[None, 1, 3, 5, 10, 20, 30],
                      'learning_rate':[0.001, 0.01, 0.1],
                      'n_estimators': [50, 100, 200]
                     }
        
        Esti_XGB = xgb.XGBClassifier(objective = 'multi:softprob')
        
        XGB_grid = GridSearchCV(estimator = Esti_XGB,
                                param_grid = xgb_params,
                                n_jobs = multiprocessing.cpu_count()-1,
                                cv = RepeatedKFold(n_splits = 5, 
                                                   n_repeats = 3, 
                                                   random_state = 42),
                                scoring='accuracy',
                                verbose = 0)
        
        CECTI_XGB = XGB_grid.fit(X_train,y_train)
        
        Best_estimador = CECTI_XGB.best_estimator_
        
        pickle.dump(Best_estimador, open("Estimador_XGB_QD.pkl", "wb"))
        
        y_hat = Best_estimador.predict(X = X_test)
        
        XG_accuracy = metrics.accuracy_score(y_hat, y_test)
        
        print("----------------------------")
        print("----------------------------")
        print ("XGBoost > Accuracy: ", np.round((XG_accuracy*100), 3), '%')
        print("----------------------------")
        print('Reporte de clasificación: \n', classification_report(y_test, y_hat))
    
    return(Best_estimador)


In [50]:
Aprendizaje_Queja_Denuncia(X_train_N, X_test_N, y_train, y_test, "Random_Forest")

----------------------------
----------------------------
Random Forest > Accuracy:  97.638 %
----------------------------
Reporte de clasificación: 
               precision    recall  f1-score   support

           0       0.99      0.97      0.98     31861
           1       0.93      0.99      0.96     12036

    accuracy                           0.98     43897
   macro avg       0.96      0.98      0.97     43897
weighted avg       0.98      0.98      0.98     43897



RandomForestClassifier(max_depth=128, max_features='auto', n_estimators=64,
                       n_jobs=7, random_state=42)